In [2]:
# start coding here
i_primary = snakemake.input.primary_aln
i_ssdis = snakemake.input.ssdis_csv
o_dataset_dir = snakemake.output.isSwitchPlus_dataset
o_full_csv = snakemake.output.isSwitchPlus_full
o_missing = snakemake.output.missing


In [3]:
from collections import Counter


from Bio import AlignIO
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


def _create_ssdis_analysis(ssdis):
    """ appends analysis columns based on input dataframe 
    
    returns:
        dataframe with secondary structure data based analysis
    
    """
    LETTERS = [
    "X",# = residues that exist in the originally studied molecule 
        # (the SEQRES records of the PDB file) 
        # but not in the observed structure (the coordinate records of the PDB file)
    "H",# = alpha helix
    "B",# = residue in isolated beta-bridge
    "E",# = extended strand, participates in beta ladder
    "G",# = 3-helix (3/10 helix)
    "I",# = 5 helix (pi helix)
    "T",# = hydrogen bonded turn
    "S",# = bend
    "L", # CUSTOM = stands for a loop or other irregular structure
    "-", # indel from alignment
    #"?" # missing in SS_DIS, not needed as these are removed early
    ]
    
    DEFINED_SECONDARY_STRUCTURE = ["H","B","E","G","I","S","L"]
    
    
    
    # remove all alignments with ? indicating SS_DIS did not contain them
    # removes any column (e.g. an aligned sequence) where the ss_dis data was missing
    #print((ssdis == '?').any())
    ssdis = ssdis.loc[:, ~(ssdis == '?').any()]
    
    for i, row in enumerate(ssdis.iterrows()):
        counts = Counter(row[1])
        cluster_size = len(row[1])
        # if cluster size drops below at least 2 after removing missing ss_dis columns
        # raise ValueError
        # if cluster_size < 2:
        #    raise ValueError("One or Zero sequences remaining after removing missing ss_dis columns")
        
        # generate counts and proportions for each secondary structure letter
        for letter in LETTERS:
            ssdis.at[i, f"{letter}_count"] = counts[letter]
            # size of the characters MINUS indel characters at the position
            # this will give proportion of residues present, ignoring alignment indels
            # set proportios to zero in first case 
            # (where all positions are indels, this occurs when an alignment like '-,-,?' occurs
            # because ? columns are removed)
            if cluster_size - counts["-"] == 0:
                ssdis.at[i, f"{letter}_proportion_of_present"] = 0
                ssdis.at[i, f"-_proportion_of_present"] = 1
            else:
                ssdis.at[i, f"{letter}_proportion_of_present"] = (counts[letter]/(cluster_size-counts["-"])) 
        
    ssdis["ClusterSize"] = cluster_size
    
    ########################################
    # START SWITCH ASSIGNMENT
    # switch categories categorized here
    # WARN: THE ORDER IS VERY IMPORTANT HERE AS LATER CATERGORIES SUPERSEDE EARLY ONES

    # default value set
    ssdis[f"isSwitch"] = "SwitchObserved"

    # set to Missing Observations
    ssdis.loc[ssdis["X_count"] > 1, "isSwitch"] = "NoSwitchObserved_WithUnobservedPositions"


    # 100% SINGLE DEFINED_SECONDARY_STRUCTURE
    for letter in DEFINED_SECONDARY_STRUCTURE:
        ssdis.loc[ssdis[f"{letter}_proportion_of_present"] == 1, "isSwitch"] = "NoSwitchObserved"
    
    
    # END SWITCH ASSIGNMENT
    ########################################
        
    return ssdis

    


def isSwitchPlus(primary_aln_path, ssdis_csv_path):
    """ Combines primary sequence data and ssdis aligned data
    
    Outputs:
        isSwitch
        percentage breakdown in secondary structures
        length of alignment (e.g. cluster size)
    
    """
    # import ssdis data and generate analysis columns
    ssdis = _create_ssdis_analysis(pd.read_csv(ssdis_csv_path))
    
    # import primary sequence alignment and append to dataframe
    aln = AlignIO.read(primary_aln_path, format="fasta")
    for seq in aln:
        try:
            ssdis[f"{seq.id}_primary"] = list(seq.seq)
        except ValueError as e:
            #print(seq.seq)
            #display(ssdis)
            raise e
    
    # DEBUG
    #display(ssdis)
    #1/0
    # END DEBUG
    
    # save full dataframe
    #ssdis.to_csv()
    
    # format final dataset style dataframes
    # these can be easily concatenated as the column names will be conserved
    ssdis_datasets = {}
    for seq in aln:
        seq_primary = f"{seq.id}_primary"
        save_these_columns = [seq_primary, "ClusterSize", "isSwitch"]
        dataset_df = ssdis.filter(axis="columns", items=save_these_columns)
        # make column label generic for primary sequence
        dataset_df = dataset_df.rename(axis="columns",mapper={seq_primary:"primary"})
        ssdis_datasets[seq.id] = dataset_df
    
    
    return ssdis_datasets, ssdis

In [8]:
import os
from pathlib import Path

issue_log = open(o_missing, "w+")


for i, primary in enumerate(i_primary):
    print(f"Running {i+1} of {len(i_primary)} : target: {primary} {' '*8}", end="\r")
    ssdis = i_ssdis[i]
    
    try:
        dataset_dfs, full_df  = isSwitchPlus(primary_aln_path=primary, ssdis_csv_path=ssdis)
    except ValueError:
        print(f"Skipped {i}")
        issue_log.write(primary + "\n")
        Path(o_full_csv[i]).touch()
        continue

    # save results to both dataset compatible csv and full cluster csv format
    for seqID, df in dataset_dfs.items():
        o_dataset_path = os.path.join(o_dataset_dir, seqID[0:2], f"{seqID}.csv")
        os.makedirs(os.path.dirname(o_dataset_path), exist_ok=True)
        df.to_csv(o_dataset_path)
    full_df.to_csv(o_full_csv[i])

    

In [ ]:
# notes
# SKIP = [675] # TRUNCATION OCCURING DURING rule: ss_dis overlay
""" excerpt: %load data/cluster.msa.secstruct/1s/1s5l_z.fasta
>4fby_Z mol:protein length:62  Photosystem II reaction center protein Z
LHHHHHHHHHHHHHHHHHHHHHHHHHHHSLSLSSSHHHHHHHHHHHHHHHHHHHHHHHHH
HL
>4fby_l mol:protein length:62  Photosystem II reaction center protein Z
LLLLTTLLLBLLLHHHHHHHHHHHHHHHHHHHHHHHL
>4il6_Z mol:protein length:62  Photosystem II reaction center protein Z
LHHHHHHHHHHHHHHHHHHHHHHHHHHTLSSSHHHHHHHHHHHHHHHHHHHHHHHHHGGG
TL
## CONFIRMED: this is due to very rare lower/uppercase sensitive chain IDs
## There is not an easy way to fix this as the bondugula IDs are all lower cased
"""

